ATTEMPT #1: 72% Accuracy
created bins for teh ask amt column and also added another hidden layer.


## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)



application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Define custom bins for ASK_AMT
custom_bins = [0, 10000, 50000, 100000, 500000, 1000000, float('inf')]  # Define the bin edges

# Define labels for the bins
bin_labels = ['0-10k', '10k-50k', '50k-100k', '100k-500k', '500k-1M', '1M+']

# Create a new column "ASK_AMT_bins" with the custom bins
application_df['ASK_AMT_bins'] = pd.cut(application_df['ASK_AMT'], bins=custom_bins, labels=bin_labels, include_lowest=True)


# Dropping ask-amt column
application_df.drop(columns=['ASK_AMT'], inplace=True)


# Check the distribution of the new bins
application_df['ASK_AMT_bins'].value_counts()


ASK_AMT_bins
0-10k        25947
10k-50k       2398
100k-500k     2304
1M+           1577
50k-100k      1423
500k-1M        650
Name: count, dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

cutoff_value = 1000

application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 1]


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

cutoff_value = 1000

classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < cutoff_value].index.tolist()


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['ASK_AMT_bins', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'], dtype=int)



application_df.head()


,STATUS,IS_SUCCESSFUL,ASK_AMT_bins_0-10k,ASK_AMT_bins_10k-50k,ASK_AMT_bins_50k-100k,ASK_AMT_bins_100k-500k,ASK_AMT_bins_500k-1M,ASK_AMT_bins_1M+,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,1,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values

X = application_df.drop(columns="IS_SUCCESSFUL").values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=45))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=23, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=11, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model (after redefining)
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3680      
                                                                 
 dense_1 (Dense)             (None, 23)                1863      
                                                                 
 dense_2 (Dense)             (None, 11)                264       
                                                                 
 dense_3 (Dense)             (None, 1)                 12        
                                                                 
Total params: 5819 (22.73 KB)
Trainable params: 5819 (22.73 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5744 - accuracy: 0.7156
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5560 - accuracy: 0.7298
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5525 - accuracy: 0.7318
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7333
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7339
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7345
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5468 - accuracy: 0.7357
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accuracy: 0.7357
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5449 - accuracy: 0.7357
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5444 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

ATTEMPT #2: 72% Accuracy

Added another hidden layer and adjusted the neuron size

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=45))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model (after redefining)
nn.summary()


nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])



In [20]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6155 - accuracy: 0.7102
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5815 - accuracy: 0.7287
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5721 - accuracy: 0.7319
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5653 - accuracy: 0.7305
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5611 - accuracy: 0.7320
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5580 - accuracy: 0.7337
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5557 - accuracy: 0.7343
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5540 - accuracy: 0.7348
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5526 - accuracy: 0.7360
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5517 - accura

In [21]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5773 - accuracy: 0.7188 - 486ms/epoch - 2ms/step
Loss: 0.5772812366485596, Accuracy: 0.7188338041305542


ATTEMPT #3: 72% Accuracy

Increasing the batch size and epochs to see if there is any improvement.

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=45))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model (after redefining)
nn.summary()


nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 80)                3680      
                                                                 
 dense_10 (Dense)            (None, 25)                2025      
                                                                 
 dense_11 (Dense)            (None, 10)                260       
                                                                 
 dense_12 (Dense)            (None, 5)                 55        
                                                                 
 dense_13 (Dense)            (None, 1)                 6         
                                                                 
Total params: 6026 (23.54 KB)
Trainable params: 6026 (23.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
batch_size = 64


fit_model = nn.fit(X_train_scaled, y_train, batch_size=batch_size, epochs=150)



Epoch 1/150
402/402 [==============================] - 1s 2ms/step - loss: 0.5477 - accuracy: 0.7330
Epoch 2/150
402/402 [==============================] - 1s 2ms/step - loss: 0.5469 - accuracy: 0.7355
Epoch 3/150
402/402 [==============================] - 1s 2ms/step - loss: 0.5460 - accuracy: 0.7340
Epoch 4/150
402/402 [==============================] - 1s 2ms/step - loss: 0.5458 - accuracy: 0.7361
Epoch 5/150
402/402 [==============================] - 1s 2ms/step - loss: 0.5450 - accuracy: 0.7358
Epoch 6/150
402/402 [==============================] - 1s 2ms/step - loss: 0.5444 - accuracy: 0.7363
Epoch 7/150
402/402 [==============================] - 1s 2ms/step - loss: 0.5437 - accuracy: 0.7373
Epoch 8/150
402/402 [==============================] - 1s 2ms/step - loss: 0.5435 - accuracy: 0.7368
Epoch 9/150
402/402 [==============================] - 1s 2ms/step - loss: 0.5431 - accuracy: 0.7379
Epoch 10/150
402/402 [==============================] - 1s 2ms/step - loss: 0.5424 - accura

In [26]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5782 - accuracy: 0.7206 - 482ms/epoch - 2ms/step
Loss: 0.5782361030578613, Accuracy: 0.7205830812454224


In [27]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the file path within your Google Drive directory
filepath = "/content/drive/My Drive/AlphabetSoupCharity_Optimization.h5"

# Export the model to HDF5 file
nn.save(filepath, save_format='h5')


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
